In [16]:
from datasets import load_dataset, Dataset
import random
from evaluate import load
import os
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import kagglehub


In [17]:
# Download latest version
path = kagglehub.dataset_download("snap/amazon-fine-food-reviews")

print("Path to dataset files:", path)

for dirname, _, filenames in os.walk('/Users/milez/.cache/kagglehub/datasets/snap/amazon-fine-food-reviews/versions/2'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Path to dataset files: /Users/milez/.cache/kagglehub/datasets/snap/amazon-fine-food-reviews/versions/2
/Users/milez/.cache/kagglehub/datasets/snap/amazon-fine-food-reviews/versions/2/database.sqlite
/Users/milez/.cache/kagglehub/datasets/snap/amazon-fine-food-reviews/versions/2/Reviews.csv
/Users/milez/.cache/kagglehub/datasets/snap/amazon-fine-food-reviews/versions/2/hashes.txt


In [18]:
df = pd.read_csv("/Users/milez/.cache/kagglehub/datasets/snap/amazon-fine-food-reviews/versions/2/" + "reviews.csv", usecols=["Id","Summary", "Text", "ProductId"])
df.dropna(subset=["Summary", "Text"], inplace=True)
df = df.sample(10000, random_state=42)
df = df.rename(columns={"Summary": "target_text", "Text": "input_text"})
dataset = Dataset.from_pandas(df)

In [19]:
tokenizer = BartTokenizer.from_pretrained("lucadiliello/bart-small")
model = BartForConditionalGeneration.from_pretrained("lucadiliello/bart-small")

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.


model.safetensors:   0%|          | 0.00/282M [00:00<?, ?B/s]

In [20]:
def preprocess_function(examples):
    return tokenizer(
        examples["input_text"],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    ) | {
        "labels": tokenizer(
            examples["target_text"],
            max_length=64,
            truncation=True,
            padding="max_length"
        )["input_ids"]
    }


In [21]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split["train"]
eval_dataset = split["test"]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [22]:
training_args = TrainingArguments(
    output_dir= "./bart_summarizer",
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir= "./logs",
    logging_steps=100,
    save_strategy="epoch",
    push_to_hub=False,
)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

model.save_pretrained("./bart_summarizer")
tokenizer.save_pretrained("./bart_summarizer")


/Users/milez/anaconda3/envs/nlp3/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/fv/_1h3qgh10y31djr_b1ns76km0000gn/T/ipykernel_21242/543969423.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.423500,0.443673
2,0.386100,0.428098
3,0.297900,0.432746


/Users/milez/anaconda3/envs/nlp3/lib/python3.10/site-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


NameError: name 'models_path' is not defined

In [23]:
def evaluate_model(model_to_eval, dataset, tokenizer, max_input_length=512, max_target_length=64):
    model_to_eval.eval()
    predictions = []
    references = []

    for example in dataset.select(range(200)):
        input_text = "summarize: " + example["input_text"]
        input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=max_input_length)
        input_ids = input_ids.to(model_to_eval.device)
        with torch.no_grad():
            output_ids = model_to_eval.generate(input_ids, max_length=max_target_length)
        pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        predictions.append(pred)
        references.append(example["target_text"])
    
    results = {}
    # ROUGE-L
    rouge_results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    results["rougeL"] = rouge_results["rougeL"]
    # BLEU score
    bleu_results = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    results["bleu"] = bleu_results["bleu"]
    # METEOR score
    meteor_results = meteor.compute(predictions=predictions, references=references)
    results["meteor"] = meteor_results["meteor"]

    return results

# Initial model
initial_results = evaluate_model(initial_model, eval_dataset, tokenizer)
print("Initial Model Metrics:")
print(f"ROUGE-L (Initial pre-trained model): {initial_results['rougeL']:.4f}")
print(f"BLEU (Initial pre-trained model): {initial_results['bleu']:.4f}")
print(f"METEOR (Initial pre-trained model): {initial_results['meteor']:.4f}")

# Fine-tuned model
finetuned_results = evaluate_model(finetuned_model, eval_dataset, tokenizer)
print("\nFine-tuned Model Metrics:")
print(f"ROUGE-L (Fine-tuned model): {finetuned_results['rougeL']:.4f}")
print(f"BLEU (Fine-tuned model): {finetuned_results['bleu']:.4f}")
print(f"METEOR (Fine-tuned model): {finetuned_results['meteor']:.4f}")

NameError: name 'initial_model' is not defined